GPU에서 실행하는 것은 CPU와의 호환성 문제가 있습니다.

    1. 딥러닝의 하이퍼 파라미터는 여러 가지가 있으며, 이를 조정하려면 몇 주 또는 몇 달이 걸립니다.
        일반적으로 연구자들은 이 튜닝을 수행하고 다른 아키텍처보다 성능이 뛰어난 아키텍처 세트를 발견하면 논문을 발표합니다.

    2. 사전 교육을 받은 모델이기 때문에 매우 빠르게 수렴되고 사용자도 GPU를 사용해야 합니다. 일부 라이브러리 문제로 인해 CPU에서 작동하지 않습니다.

    3. 우리의 목적을 위해 우리는 연구원들이 이용할 수 있는 그런 아키텍처를 이용할 수 있습니다.

    4. 이미 기능을 추출하는 방법을 알고 있는 미리 훈련된 네트워크를 사용하면 하이퍼 파라미터를 조정할 필요가 없습니다. 이미 일부 데이터 세트(이미지넷이라고 함)에 대해 교육을 받았기 때문에 미리 훈련된 가중치는 가중치의 초기화를 잘 제공하며, 이로 인해 우리의 Convnet은 매우 빠르게 수렴되며 그렇지 않으면 이러한 심층 아키텍처에 며칠이 걸릴 수 있습니다. 그것이 Transfer Learning의 아이디어 입니다. 그 예로는 VGG16, InceptionNet, google, Resnet 등이 있습니다.

    5. 이 커널에서는 작은 크기의 이미지에서 매우 우수한 성능을 발휘하는 사전 훈련된 VGG16 네트워크를 사용할 것입니다.

VGG 아키텍처가 작은 크기의 이미지에서도 잘 작동하는 것으로 입증되었습니다.(CIFAR-10) 이 데이터 세트에서도 잘 작동할 것으로 예상했습니다.

이 코드에는 데이터 확대 단계도 포함되어 있으므로 성능이 상당히 개선됩니다.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import log_loss
from os.path import join as opj
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [2]:
train = pd.read_json('./input/train.json')
test = pd.read_json('./input/test.json')
target_train = train['is_iceberg']

Keras는 사전 훈련된 VGG의 구현을 제공합니다. 그것은 라이브러리에 있으므로 우리는 직접 네트워크를 구축할 필요가 없습니다. 여기서는 VGG의 마지막 레이어를 제거하고 이진 예측을 위한 시그모이드 레이어를 배치합니다.

In [9]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)



X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

#Import Keras.
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input	

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator
batch_size=64
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
   es = EarlyStopping('val_loss', patience=10, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]


def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model([base_model.input, input_2], predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model


#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [10]:
preds = myAngleCV(X_train, X_angle, X_test)


===================FOLD= 0


C:\Users\유재성\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
24/24 [==============================] - 80s 3s/step - loss: 1.0372 - accuracy: 0.5424 - val_loss: 0.5274 - val_accuracy: 0.7308
Epoch 2/100
24/24 [==============================] - 77s 3s/step - loss: 0.5759 - accuracy: 0.7171 - val_loss: 0.3283 - val_accuracy: 0.8411
Epoch 3/100
24/24 [==============================] - 79s 3s/step - loss: 0.4005 - accuracy: 0.8183 - val_loss: 0.3549 - val_accuracy: 0.8299
Epoch 4/100
24/24 [==============================] - 78s 3s/step - loss: 0.3196 - accuracy: 0.8636 - val_loss: 0.2951 - val_accuracy: 0.8729
Epoch 5/100
24/24 [==============================] - 78s 3s/step - loss: 0.2809 - accuracy: 0.8644 - val_loss: 0.2547 - val_accuracy: 0.8879
Epoch 6/100
24/24 [==============================] - 78s 3s/step - loss: 0.2854 - accuracy: 0.8765 - val_loss: 0.2688 - val_accuracy: 0.8991
Epoch 7/100
24/24 [==============================] - 78s 3s/step - loss: 0.2473 - accuracy: 0.8976 - val_loss: 0.2270 - val_accuracy: 0.9103
Epoch 8/100
2

In [11]:
# Submission for each day
submission = pd.DataFrame()
submission['id'] = test['id']
submission['is_iceberg'] = preds
submission.to_csv('sub.csv', index=False)

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())


In [1]:

import numpy as np

import pandas as pd



import keras

import tensorflow as tf

from IPython.display import display

import PIL


In [4]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8208108715457216335
]
